In [8]:
import pandas as pd
import requests, zipfile, io
!pip install pyxlsb
import pyxlsb
from IPython.core.display import HTML

import warnings
warnings.filterwarnings('ignore')

# Set up some default parameters for graphing
import matplotlib.pyplot as plt
from matplotlib import cycler
colour = "#00C2AB" # The default colour for the barcharts
colors = cycler('color', ['#4FBBA9', '#E56D13', '#D43A69','#25539f', '#88BB44', '#FFBBBB'])
plt.rc('axes', facecolor='#E6E6E6', edgecolor='none',
       axisbelow=True, grid=True, prop_cycle=colors)
plt.rc('grid', color='w', linestyle='solid')
plt.rc('xtick', direction='out', color='gray')
plt.rc('ytick', direction='out', color='gray')
plt.rc('patch', edgecolor='#E6E6E6')
plt.rc('lines', linewidth=2)
font = {'family' : 'DejaVu Sans',
        'weight' : 'normal',
        'size'   : 16}
plt.rc('font', **font)
plt.rc('legend', fancybox = True, framealpha=1, shadow=True, borderpad=1)

In [10]:
# We will load in the sheets in the unzipped XLSX file that contain various kinds of direct emissions, from individual facilities and sectors/activities
ghg_fac = pd.read_excel("ghgp_data_2020.xlsx",
  skiprows = 1, # skip the first two rows, which are irrelevant
  header = 2, # the third row represents the header
  converters={'Primary NAICS Code':str}, # read the NAICS codes as strings
  sheet_name = ["Direct Emitters", "Onshore Oil & Gas Prod.", "Gathering & Boosting", 
    "Transmission Pipelines", "LDC - Direct Emissions", "SF6 from Elec. Equip.", "Suppliers"]) 

# We will also load in a file that tells us about the type of industry each facility is classified as
naics = pd.read_excel("https://www.census.gov/naics/2017NAICS/6-digit_2017_Codes.xlsx",
  header = 0, # the row that represents the header
  converters={'2017 NAICS Code':str,'2017 NAICS Title':str}
)
naics=naics[["2017 NAICS Code", "2017 NAICS Title"]]

In [11]:
ghg_fac["Suppliers"]["Emissions (tCO2e)"] = pd.to_numeric(ghg_fac["Suppliers"]["GHG Quantity Associated with Coal-based liquid fuel production"], errors='coerce').fillna(0) + \
  pd.to_numeric(ghg_fac["Suppliers"]["GHG Quantity Associated with Petroleum Products Produced"], errors='coerce').fillna(0) + \
  pd.to_numeric(ghg_fac["Suppliers"]["GHG Quantity Associated with Petroleum Products Imported"], errors='coerce').fillna(0) + \
  pd.to_numeric(ghg_fac["Suppliers"]["GHG Quantity Associated with Petroleum Products Exported"], errors='coerce').fillna(0) + \
  pd.to_numeric(ghg_fac["Suppliers"]["GHG Quantity Associated with Natural Gas Supply"], errors='coerce').fillna(0) + \
  pd.to_numeric(ghg_fac["Suppliers"]["GHG Quantity Associated with Natural Gas Liquids Supply"], errors='coerce').fillna(0) + \
  pd.to_numeric(ghg_fac["Suppliers"]["GHG Quantity Associated with CO2 Supply "], errors='coerce').fillna(0)

emissions_field_names = {
    "Direct Emitters": "Total reported direct emissions",
    "Onshore Oil & Gas Prod.": "Total reported emissions from Onshore Oil & Gas Production ",
    "Gathering & Boosting": "Total reported emissions from Gathering & Boosting",
    "Transmission Pipelines": "Total reported direct emissions from Transmission Pipelines",
    "LDC - Direct Emissions": "Total reported direct emissions from Local Distribution Companies",
    "SF6 from Elec. Equip.": "Total reported direct emissions from Electrical Equipment Use"
}

fac_directemitters_data = pd.DataFrame([], columns=["Facility", "Name", "Primary NAICS Code", "Emissions (tCO2e)"]) 
fac_suppliers_data = pd.DataFrame(ghg_fac["Suppliers"]) 

for key in emissions_field_names.keys():
  dump = ghg_fac[key]
  dump = dump[["Facility Id", "Facility Name", "Primary NAICS Code", emissions_field_names[key]]]
  dump = dump.rename(columns={"Facility Id": "Facility", "Facility Name": "Name", "Primary NAICS Code": "2017 NAICS Code", emissions_field_names[key]:"Emissions (tCO2e)"})
  dump["Emissions (tCO2e)"] = pd.to_numeric(dump["Emissions (tCO2e)"])
  fac_directemitters_data= fac_directemitters_data.append(dump)

fac_directemitters_data = fac_directemitters_data.groupby(by=["Facility", "Name", "2017 NAICS Code"]).sum().reset_index() #For presentation purposes, we'll add in the facility name
fac_suppliers_data=fac_suppliers_data.rename(columns={"Facility Id": "Facility", "Facility Name": "Name", "Primary NAICS Code": "2017 NAICS Code"})
fac_suppliers_data = fac_suppliers_data[["Facility", "Name", "2017 NAICS Code", "Emissions (tCO2e)"]]
fac_directemitters_data['2017 NAICS Code']=fac_directemitters_data['2017 NAICS Code'].astype(str)

# Look up industry information
fac_directemitters_data = fac_directemitters_data.merge(naics, on="2017 NAICS Code")
fac_suppliers_data = fac_suppliers_data.merge(naics, on="2017 NAICS Code")


In [13]:
url = "https://www.epa.gov/system/files/other-files/2021-10/ghgp_data_parent_company_10_2021.xlsb"
parents = pd.read_excel(url, engine='pyxlsb', sheet_name="2020", nrows=9144) # 9144 non-header rows in this file as of Nov 2021.
parents["PARENT COMPANY NAME"] = parents["PARENT COMPANY NAME"].str.upper() # Standardize company names to uppercase - sometimes EPA allows company names to be reported in upper and lower case, which intereferes with our ability to aggregate them
parents

,GHGRP FACILITY ID,FRS ID (FACILITY),REPORTING YEAR,FACILITY NAME,FACILITY ADDRESS,FACILITY CITY,FACILITY STATE,FACILITY ZIP,FACILITY COUNTY,PARENT COMPANY NAME,PARENT CO. STREET ADDRESS,PARENT CO. CITY,PARENT CO. STATE,PARENT CO. ZIP,PARENT CO. PERCENT OWNERSHIP,FACILITY NAICS CODE
0,1000001,1.100005e+11,2020,PSE Ferndale Generating Station,5105 LAKE TERRELL ROAD,FERNDALE,WA,98248,WHATCOM COUNTY,PUGET HOLDINGS LLC,PO Box 97034,Bellevue,WA,98009,100.0,221112
1,1000002,1.100412e+11,2020,Ardagh Glass Inc. (Dunkirk),524 E. CENTER STREET,DUNKIRK,IN,47336,JAY COUNTY,ARDAGH GLASS INC,1209 Orange Street,Wilmington,DE,19801,100.0,327213
2,1000003,1.100015e+11,2020,Ardagh Glass Inc. (Henderson),620 Facet Road,Henderson,NC,27537,VANCE COUNTY,ARDAGH GLASS INC,1209 Orange Street,Wilmington,DE,19801,100.0,327213
3,1000005,1.100005e+11,2020,Ardagh Glass Inc. (Madera),24441 AVENUE 12 & ROAD 24 1/2,MADERA,CA,93637,MADERA COUNTY,ARDAGH GLASS INC,1209 Orange Street,Wilmington,DE,19801,100.0,327213
4,1000007,1.100070e+11,2020,AMERICAN GYPSUM - BERNALILLO (WALLBOARD) PLANT,1000 N HILL RD,BERNALILLO,NM,87004,SANDOVAL COUNTY,EAGLE MATERIALS INC,5960 Berkshire Lane Suite 800,Dallas,TX,75225,100.0,327420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9139,1014033,NaN,2020,AW PRODUCT SALES AND MARKETING INC,"204 E. 2nd Avenue, Unit #343",San Mateo,CA,94401,NaN,AW DISTRIBUTING INC,"204 E. 2nd Avenue, Unit #343",San Mateo,CA,94401,100.0,522293
9140,1014041,NaN,2020,"Bluon, Inc.",18 Bunsen,Irvine,CA,92618,NaN,BLUON INC,18 Bunsen,Irvine,CA,92618,100.0,424690
9141,1014043,NaN,2020,RAMJ Enterprises Inc,4912 W KNOX ST STE 100,Tampa,FL,33634,NaN,RAMJ ENTERPRISES INC,4912 W KNOX ST,Tampa,FL,33634,100.0,425120
9142,1014044,1.100626e+11,2020,Wilhelmsen Ships Service Inc,9400 New Century Dr,Pasadena,TX,77507,HARRIS COUNTY,WILHELMSEN SHIPS SERVICE,9400 New Century Dr.,Pasadena,TX,77507,100.0,484220


In [14]:
join_emitters = fac_directemitters_data.set_index("Facility").join(parents.set_index('GHGRP FACILITY ID'))
join_suppliers = fac_suppliers_data.set_index("Facility").join(parents.set_index('GHGRP FACILITY ID'))

In [15]:
join_suppliers

,Name,2017 NAICS Code,Emissions (tCO2e),2017 NAICS Title,FRS ID (FACILITY),REPORTING YEAR,FACILITY NAME,FACILITY ADDRESS,FACILITY CITY,FACILITY STATE,FACILITY ZIP,FACILITY COUNTY,PARENT COMPANY NAME,PARENT CO. STREET ADDRESS,PARENT CO. CITY,PARENT CO. STATE,PARENT CO. ZIP,PARENT CO. PERCENT OWNERSHIP,FACILITY NAICS CODE
1000008,"St. Lawrence Gas Co., Inc.",221210,3.786518e+05,Natural Gas Distribution,1.100701e+11,2020,"St. Lawrence Gas Co., Inc.",33 Stearns Street,Massena,NY,13662,ST. LAWRENCE COUNTY,ST LAWRENCE GAS CO,33 Stearns Street,Massena,NY,13662,100.0,221210
1000011,Central Hudson Gas and Electric Corporation,221122,1.011668e+06,Electric Power Distribution,1.100008e+11,2020,Central Hudson Gas and Electric Corporation,284 South Avenue,Poughkeepsie,NY,12601,DUTCHESS COUNTY,CENTRAL HUDSON GAS & ELECTRIC CORP,284 South Avenue,Poughkeepsie,NY,12601,100.0,221122
1000025,PLACID REFINING CO LLC - PORT ALLEN REFINERY,324110,9.076471e+06,Petroleum Refineries,1.100004e+11,2020,PLACID REFINING CO LLC - PORT ALLEN REFINERY,"1940 LA HWY 1, NORTH",PORT ALLEN,LA,70767,WEST BATON ROUGE PARISH,PLACID REFINING CO LLC,1940 LA HWY 1 North,Port Allen,LA,70767,100.0,324110
1000035,THE ANDERSONS MARATHON HOLDINGS LLC,325193,0.000000e+00,Ethyl Alcohol Manufacturing,1.100376e+11,2020,THE ANDERSONS MARATHON HOLDINGS LLC,5728 E SEBRING WARNER RD,GREENVILLE,OH,45331,DARKE COUNTY,THE ANDERSONS INC,PO Box 119,Maumee,OH,43537,50.0,325193
1000035,THE ANDERSONS MARATHON HOLDINGS LLC,325193,0.000000e+00,Ethyl Alcohol Manufacturing,1.100376e+11,2020,THE ANDERSONS MARATHON HOLDINGS LLC,5728 E SEBRING WARNER RD,GREENVILLE,OH,45331,DARKE COUNTY,MARATHON PETROLEUM CORP,539 South Main Street,Findlay,OH,45840,25.0,325193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014033,AW PRODUCT SALES AND MARKETING INC,522293,0.000000e+00,International Trade Financing,NaN,2020,AW PRODUCT SALES AND MARKETING INC,"204 E. 2nd Avenue, Unit #343",San Mateo,CA,94401,NaN,AW DISTRIBUTING INC,"204 E. 2nd Avenue, Unit #343",San Mateo,CA,94401,100.0,522293
1014041,"Bluon, Inc.",424690,0.000000e+00,Other Chemical and Allied Products Merchant Wh...,NaN,2020,"Bluon, Inc.",18 Bunsen,Irvine,CA,92618,NaN,BLUON INC,18 Bunsen,Irvine,CA,92618,100.0,424690
1014043,RAMJ Enterprises Inc,425120,0.000000e+00,Wholesale Trade Agents and Brokers,NaN,2020,RAMJ Enterprises Inc,4912 W KNOX ST STE 100,Tampa,FL,33634,NaN,RAMJ ENTERPRISES INC,4912 W KNOX ST,Tampa,FL,33634,100.0,425120
1014044,Wilhelmsen Ships Service Inc,484220,0.000000e+00,Specialized Freight (except Used Goods) Trucki...,1.100626e+11,2020,Wilhelmsen Ships Service Inc,9400 New Century Dr,Pasadena,TX,77507,HARRIS COUNTY,WILHELMSEN SHIPS SERVICE,9400 New Century Dr.,Pasadena,TX,77507,100.0,484220


In [16]:
nas_s = join_suppliers.loc[join_suppliers["Emissions (tCO2e)"]  == 0]

In [17]:
nas_s

,Name,2017 NAICS Code,Emissions (tCO2e),2017 NAICS Title,FRS ID (FACILITY),REPORTING YEAR,FACILITY NAME,FACILITY ADDRESS,FACILITY CITY,FACILITY STATE,FACILITY ZIP,FACILITY COUNTY,PARENT COMPANY NAME,PARENT CO. STREET ADDRESS,PARENT CO. CITY,PARENT CO. STATE,PARENT CO. ZIP,PARENT CO. PERCENT OWNERSHIP,FACILITY NAICS CODE
1000035,THE ANDERSONS MARATHON HOLDINGS LLC,325193,0.0,Ethyl Alcohol Manufacturing,1.100376e+11,2020,THE ANDERSONS MARATHON HOLDINGS LLC,5728 E SEBRING WARNER RD,GREENVILLE,OH,45331,DARKE COUNTY,THE ANDERSONS INC,PO Box 119,Maumee,OH,43537,50.0,325193
1000035,THE ANDERSONS MARATHON HOLDINGS LLC,325193,0.0,Ethyl Alcohol Manufacturing,1.100376e+11,2020,THE ANDERSONS MARATHON HOLDINGS LLC,5728 E SEBRING WARNER RD,GREENVILLE,OH,45331,DARKE COUNTY,MARATHON PETROLEUM CORP,539 South Main Street,Findlay,OH,45840,25.0,325193
1000035,THE ANDERSONS MARATHON HOLDINGS LLC,325193,0.0,Ethyl Alcohol Manufacturing,1.100376e+11,2020,THE ANDERSONS MARATHON HOLDINGS LLC,5728 E SEBRING WARNER RD,GREENVILLE,OH,45331,DARKE COUNTY,MARATHON PETROLEUM CORP,539 South Main Street,Findlay,OH,45840,25.0,325193
1000039,GE's Grid Solutions,335313,0.0,Switchgear and Switchboard Apparatus Manufactu...,1.100329e+11,2020,GE's Grid Solutions,One Power Lane,Charleroi,PA,15022,WASHINGTON COUNTY,GENERAL ELECTRIC CO,41 Farnsworth Street,Boston,MA,2210,100.0,335313
1000158,Ascend (Decatur Plant),325199,0.0,All Other Basic Organic Chemical Manufacturing,1.100072e+11,2020,Ascend (Decatur Plant),1050 CHEMSTRAND AVENUE,DECATUR,AL,35601,Morgan,ASCEND PERFORMANCE MATERIALS HOLDINGS INC,"1010 Travis Street, Suite 900",Houston,TX,77002-3014,100.0,325199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014033,AW PRODUCT SALES AND MARKETING INC,522293,0.0,International Trade Financing,NaN,2020,AW PRODUCT SALES AND MARKETING INC,"204 E. 2nd Avenue, Unit #343",San Mateo,CA,94401,NaN,AW DISTRIBUTING INC,"204 E. 2nd Avenue, Unit #343",San Mateo,CA,94401,100.0,522293
1014041,"Bluon, Inc.",424690,0.0,Other Chemical and Allied Products Merchant Wh...,NaN,2020,"Bluon, Inc.",18 Bunsen,Irvine,CA,92618,NaN,BLUON INC,18 Bunsen,Irvine,CA,92618,100.0,424690
1014043,RAMJ Enterprises Inc,425120,0.0,Wholesale Trade Agents and Brokers,NaN,2020,RAMJ Enterprises Inc,4912 W KNOX ST STE 100,Tampa,FL,33634,NaN,RAMJ ENTERPRISES INC,4912 W KNOX ST,Tampa,FL,33634,100.0,425120
1014044,Wilhelmsen Ships Service Inc,484220,0.0,Specialized Freight (except Used Goods) Trucki...,1.100626e+11,2020,Wilhelmsen Ships Service Inc,9400 New Century Dr,Pasadena,TX,77507,HARRIS COUNTY,WILHELMSEN SHIPS SERVICE,9400 New Century Dr.,Pasadena,TX,77507,100.0,484220


In [18]:
nas_s['PARENT COMPANY NAME']

1000035                             THE ANDERSONS INC
1000035                       MARATHON PETROLEUM CORP
1000035                       MARATHON PETROLEUM CORP
1000039                           GENERAL ELECTRIC CO
1000158     ASCEND PERFORMANCE MATERIALS HOLDINGS INC
                              ...                    
1014033                           AW DISTRIBUTING INC
1014041                                     BLUON INC
1014043                          RAMJ ENTERPRISES INC
1014044                      WILHELMSEN SHIPS SERVICE
1014063    TRANSOCEAN OFFSHORE DEEPWATER DRILLING INC
Name: PARENT COMPANY NAME, Length: 364, dtype: object

In [20]:
join_emitters["PARENT EMISSIONS AMT"] = join_emitters['Emissions (tCO2e)'] * join_emitters["PARENT CO. PERCENT OWNERSHIP"] / 100 
join_suppliers["PARENT EMISSIONS AMT"] = join_suppliers['Emissions (tCO2e)'] * join_suppliers["PARENT CO. PERCENT OWNERSHIP"] / 100 
summary_e = join_emitters.groupby("PARENT COMPANY NAME")[["PARENT EMISSIONS AMT"]].agg(['sum', 'count'])
summary_s = join_suppliers.groupby("PARENT COMPANY NAME")[["PARENT EMISSIONS AMT"]].agg(['sum', 'count'])

In [22]:
join_emitters

,Name,2017 NAICS Code,Emissions (tCO2e),2017 NAICS Title,FRS ID (FACILITY),REPORTING YEAR,FACILITY NAME,FACILITY ADDRESS,FACILITY CITY,FACILITY STATE,FACILITY ZIP,FACILITY COUNTY,PARENT COMPANY NAME,PARENT CO. STREET ADDRESS,PARENT CO. CITY,PARENT CO. STATE,PARENT CO. ZIP,PARENT CO. PERCENT OWNERSHIP,FACILITY NAICS CODE,PARENT EMISSIONS AMT
1000001,PSE Ferndale Generating Station,221112,509154.612,Fossil Fuel Electric Power Generation,1.100005e+11,2020,PSE Ferndale Generating Station,5105 LAKE TERRELL ROAD,FERNDALE,WA,98248,WHATCOM COUNTY,PUGET HOLDINGS LLC,PO Box 97034,Bellevue,WA,98009,100.0,221112,509154.612
1000002,Ardagh Glass Inc. (Dunkirk),327213,113519.804,Glass Container Manufacturing,1.100412e+11,2020,Ardagh Glass Inc. (Dunkirk),524 E. CENTER STREET,DUNKIRK,IN,47336,JAY COUNTY,ARDAGH GLASS INC,1209 Orange Street,Wilmington,DE,19801,100.0,327213,113519.804
1000003,Ardagh Glass Inc. (Henderson),327213,81110.872,Glass Container Manufacturing,1.100015e+11,2020,Ardagh Glass Inc. (Henderson),620 Facet Road,Henderson,NC,27537,VANCE COUNTY,ARDAGH GLASS INC,1209 Orange Street,Wilmington,DE,19801,100.0,327213,81110.872
1000005,Ardagh Glass Inc. (Madera),327213,79854.076,Glass Container Manufacturing,1.100005e+11,2020,Ardagh Glass Inc. (Madera),24441 AVENUE 12 & ROAD 24 1/2,MADERA,CA,93637,MADERA COUNTY,ARDAGH GLASS INC,1209 Orange Street,Wilmington,DE,19801,100.0,327213,79854.076
1000007,AMERICAN GYPSUM - BERNALILLO (WALLBOARD) PLANT,327420,14084.496,Gypsum Product Manufacturing,1.100070e+11,2020,AMERICAN GYPSUM - BERNALILLO (WALLBOARD) PLANT,1000 N HILL RD,BERNALILLO,NM,87004,SANDOVAL COUNTY,EAGLE MATERIALS INC,5960 Berkshire Lane Suite 800,Dallas,TX,75225,100.0,327420,14084.496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1014015,Cedar Grove #2 Mine,212112,19219.500,Bituminous Coal Underground Mining,NaN,2020,Cedar Grove #2 Mine,"County Route 14, Rum Creek Road",Yolyn,WV,25654,NaN,ARACOMA COAL CO LLC,PO Box 1098,Holden,WV,25625,100.0,212112,19219.500
1014021,"Peco Foods, Inc.",311615,87368.842,Poultry Processing,1.100674e+11,2020,"Peco Foods, Inc.",4114 Hwy 67 South,Pocahontas,AR,72455,RANDOLPH COUNTY,PECO FOODS INC,1101 Greensboro Ave,Tuscaloosa,AL,35401,100.0,311615,87368.842
1014023,JSW Steel (USA) Inc.,331110,43100.388,Iron and Steel Mills and Ferroalloy Manufactur...,1.100007e+11,2020,JSW Steel (USA) Inc.,5200 E. McKinney Road,Baytown,TX,77523,CHAMBERS COUNTY,JSW STEEL USA INC,5200 East McKinney Road,Baytown,TX,77520,100.0,331110,43100.388
1014024,"TITUS OIL & GAS PRODUCTION, LLC",211120,53440.912,Crude Petroleum Extraction,NaN,2020,"TITUS OIL & GAS PRODUCTION, LLC",420 Throckmorton St. Ste 1150,Fort Worth,TX,76102,NaN,TITUS OIL & GAS,"420 Throckmorton St, Suite 1150",Fort Worth,TX,76102,100.0,211120,53440.912
